In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from gene_id import Gene_IDs
from atac_signal import ATAC_signal
from gene_sets import Gene_sets

import plotting as my_plots
import utilities as ut
import calc_signals as cas

In [ ]:
if "gs" not in locals():
    gs = Gene_sets()

if "exp1" not in locals():
    exp1 = ATAC_signal("exp1")

if "exp_mss" not in locals():
    exp_mss = ATAC_signal("exp_metsetset")

if "exp_hrde1" not in locals():
    exp_hrde1 = ATAC_signal("exp_hrde_guy")

In [ ]:
my_plots.plot_groups_signals(exp1, var_type='none')

In [ ]:
my_plots.plot_groups_signals(exp_hrde1, var_type='none')

In [ ]:
my_plots.plot_groups_signals(exp_mss, var_type='none')

In [ ]:
my_plots.plot_signal_gene(exp1, 'oma-1', plot_range=(-750, 500), var_type='sem', mean_flag=False)

In [ ]:
my_plots.plot_signal_gene(exp_mss, 'GFP', plot_range=(-750, 500), var_type='sem', mean_flag=False)

In [ ]:
my_plots.plot_gene_atac_signal_distribution(exp_mss, 'oma-1', mean_flag=True)

In [ ]:
## hrde-1 lists:
hrde1_kennedy = gs.get_list('hrde-1-Kennedy')
hrde_FC_sig = gs.get_list('mRNA_isSig')
hrde_up = gs.get_list('mRNA_log2_FC', thresh=0)
hrde_up_sig = ut.intersect_lists(hrde_FC_sig, hrde_up)
hrde_down = gs.get_list('mRNA_log2_FC', thresh=0, bottom=True)
hrde_down_sig = ut.intersect_lists(hrde_FC_sig, hrde_down)
hrde_regulated = ut.intersect_lists(hrde_up_sig, hrde1_kennedy)

hrde1_nearby_up, hrde1_nearby_down = ut.get_nearby_genes_list(hrde_regulated, 2000) # len 75, len 28
hrde1_nearby_up_1200, hrde1_nearby_down_1200 = ut.get_nearby_genes_list(hrde_regulated, 1200) # len 49, len 7

hrde_dic = {'hrde1_kennedy':hrde1_kennedy, 'hrde_reg':hrde_regulated, 'hrde-1 upstream':hrde1_nearby_up, 'hrde down sig':hrde_down_sig}
    

These genes were found by Itamar to be...

In [ ]:
my_plots.plot_gene_atac_signal_distribution(exp1, 'WBGene00015351', mean_flag=False)

In [ ]:
my_plots.plot_gene_atac_signal_distribution(exp1, 'WBGene00016177', mean_flag=False)

In [ ]:
my_plots.plot_gene_atac_signal_distribution(exp1, 'WBGene00000224', mean_flag=False)

In [ ]:
my_plots.plot_gene_atac_signal_distribution(exp1, 'WBGene00013100', mean_flag=False)

In [ ]:
cas.bootstrap_group_score_fc_histogram(exp1.fc['rep 0'], hrde1_nearby_up) # ~99%

In [ ]:
cas.bootstrap_group_score_fc_histogram(exp1.fc['rep 0'], hrde_regulated) #

In [ ]:
cas.bootstrap_group_score_fc_histogram(exp1.fc['rep 1'], hrde1_nearby_up) # ~1%

In [ ]:
cas.bootstrap_group_score_fc_histogram(exp1.fc['rep 1'], hrde_regulated) #

In [ ]:
cas.bootstrap_group_score_fc_histogram(exp1.fc['rep 2'], hrde1_nearby_up) # 5%

In [ ]:
cas.bootstrap_group_score_fc_histogram(exp1.fc['rep 3'], hrde1_nearby_up) # 67%

In [ ]:
cas.bootstrap_group_score_fc_histogram(exp1.fc.mean(axis=1), hrde1_nearby_up) # 17%

Until now, it was all testing for exp1. Now it is for exp_hrde1, which is supposed to be relevant:

In [ ]:
my_plots.plot_groups_signals(exp_hrde1, groups_dic={'hrde reg':hrde_regulated}, mean_flag=True)

In [ ]:
my_plots.plot_groups_signals(exp_hrde1, groups_dic={'hrde reg':hrde_regulated}, mean_flag=False, var_type='none')

In [ ]:
cas.bootstrap_group_score_fc_histogram(exp_hrde1.fc.mean(axis=1), hrde_regulated) # 0%

In [ ]:
cas.bootstrap_group_score_fc_histogram(exp_hrde1.fc.mean(axis=1), hrde1_nearby_up) # 47%

This looks like the "hrde1_upstream" group has a pretty standard fold_change score! not good for us.

testthe function with random data:

In [ ]:
genes_ind = exp1.fc.index
genes_cols = exp1.fc.columns
nums = np.random.rand(len(genes_ind),4)
rand_data = pd.DataFrame(nums, index = genes_ind, columns =genes_cols)

In [ ]:
cas.bootstrap_group_score_fc_histogram(rand_data.mean(axis=1), hrde1_nearby_up)

In [ ]:
intersected_list = list(set(rand_data.index) & set(hrde1_nearby_up))
rand_data.loc[intersected_list,:]=0.5
cas.bootstrap_group_score_fc_histogram(rand_data.mean(axis=1), hrde1_nearby_up)

Biological questions:

In [ ]:
exp_hrde1.exp_df

In [ ]:
import matplotlib.pyplot as plt 
ds = [1000, 1500, 2000, 3000, 5000, 10000, 15_000, 25_000]
for distance in ds:
    genes_up, genes_down = ut.get_nearby_genes_list(hrde_regulated, distance)
    print(f'for distance: {distance}, num  of genes:{len(genes_down)}')
    cas.bootstrap_group_score_fc_histogram(exp_hrde1.fc.mean(axis=1), genes_down)
    plt.show()



In [ ]:
import matplotlib.pyplot as plt 
ds = [1000, 1500, 2000, 3000, 5000, 10000, 15_000, 25_000]
for distance in ds:
    genes_up, genes_down = ut.get_nearby_genes_list(hrde_regulated, distance)
    print(f'for distance: {distance}, num  of genes:{len(genes_up)}')
    cas.bootstrap_group_score_fc_histogram(exp_hrde1.fc.mean(axis=1), genes_up)
    plt.show()

In [ ]:
genes_up_10000, genes_down_10000 = ut.get_nearby_genes_list(hrde_regulated, 10_000)
my_plots.plot_groups_signals(exp_hrde1, groups_dic={'hrde-1 downstream':genes_down_10000, 'hrde reg':hrde_regulated}, mean_flag=True)


In [ ]:
len(genes_down_10000)

In [ ]:
len(hrde_up_sig)

In [ ]:
hrde_up_and_nearby_downstream_10000 = ut.intersect_lists(genes_down_10000, hrde_up_sig)

In [ ]:
len(hrde_up_and_nearby_downstream_10000)

In [ ]:
mrna_fc = gs.big_table['mRNA_log2_FC']

In [ ]:
cas.bootstrap_group_score_fc_histogram(mrna_fc, genes_down_10000)

By stander of genes:



In [ ]:
my_plots.plot_gene_atac_signal_distribution(exp_mss, 'spr-2', mean_flag=True)

In [ ]:
genes_up_15, genes_down_15 = ut.get_nearby_genes_list(hrde_regulated, 15_000)
i_bootstrap_means, _ = cas.bootstrap_group_score(exp_hrde1.fc.mean(axis=1), genes_down_15)



In [ ]:
v_line_mean = i_bootstrap_means.mean()

In [ ]:
ds = [1500, 3000, 5000, 15_000, 25_000]
for distance in ds:
    genes_up, genes_down = ut.get_nearby_genes_list(hrde_regulated, distance)
    intersected_list = ut.intersect_lists(exp_hrde1.fc.index, genes_down)
    exp_hrde1.fc.loc[intersected_list,:].mean(axis=1).hist(bins=15, alpha=0.5, density=True)

plt.suptitle('hrde-1 nearby downstream geans')
plt.xlabel('fold-change in atac signal')
plt.ylabel('probability density (genes)')
plt.legend(ds)  
plt.vlines(v_line_mean, ymin=0, ymax=4, linestyles='dashed')


In [ ]:
intersected_list_15 = ut.intersect_lists(exp_hrde1.fc.index, genes_down_15)
exp_hrde1.fc.loc[intersected_list_15,:].mean(axis=1).hist(bins=15, alpha=0.5, density=True)

plt.suptitle('hrde-1 nearby downstream geans (15kb distance)')
plt.xlabel('fold-change in atac signal')
plt.ylabel('probability density (genes)')
plt.legend(['15kb'])  
plt.vlines(v_line_mean, ymin=0, ymax=1.5, linestyles='dashed')

In [ ]:
my_plots.plot_groups_signals(exp1, var_type='none')